# Forecasting a Time Series 


### Data loading & Processing 

In [ ]:
%matplotlib inline
import pandas as pd


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)


data = pd.read_csv("sample.csv",index_col=0)
data.head()

In [ ]:
data.index

In [ ]:
data.index = pd.to_datetime(data.index)

In [ ]:
data.head()

In [ ]:
data.index

In [ ]:
data[pd.isnull(data['IPG2211A2N'])]

In [ ]:
data.columns = ['Value']

In [ ]:
data.tail()

### ARIMA Time Series Model Setting 

In [ ]:
import itertools

# S_period: periodicity of the time series (4 for quarterly periods, 12 for yearly periods for monthly data, etc.)
S_period = 12

# Define the p, d and q parameters to take any value between 0 and 2
#p = d = q = range(0, 5)

p = range(0, 2)
d = range(0, 2)
q = range(0, 2)

# Generate all different combinations of p, q and q triplets
pdq = list(itertools.product(p, d, q))

# Generate all different combinations of seasonal p, q and q triplets
seasonal_pdq = [(x[0], x[1], x[2], S_period) for x in list(itertools.product(p, d, q))]

In [ ]:
pdq

###  Parameter Selection for the ARIMA Time Series Model

In [ ]:
import warnings
import statsmodels.api as sm
import numpy as np
import time


warnings.filterwarnings("ignore") # specify to ignore warning messages


cnt_loop = 0


aic_table = pd.DataFrame(np.zeros((len(pdq), len(seasonal_pdq))), index = pdq, columns = seasonal_pdq)

print("Time mearuse started.")
startTime = time.time()
                         
for param in pdq:
    for param_seasonal in seasonal_pdq:

        mod = sm.tsa.statespace.SARIMAX(data,
                                        order=param,
                                        seasonal_order=param_seasonal,
                                        enforce_stationarity=False,
                                        enforce_invertibility=False)

        results = mod.fit()


        print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))

        aic_table[param, param_seasonal] = results.aic

        if cnt_loop == 0:
            min_aic = results.aic
            min_aic_param = param
            min_aic_param_seasonal = param_seasonal

        if results.aic < min_aic:
            min_aic = results.aic
            min_aic_param = param
            min_aic_param_seasonal = param_seasonal          
        cnt_loop = cnt_loop + 1             

            


endTime = time.time() - startTime
print("Spent time:", endTime) 

print("Minimum AIC value:", min_aic) 

In [ ]:
min_combination = {"min_aic":min_aic, "min_aic_param":min_aic_param, "min_aic_param_seasonal":min_aic_param_seasonal}


print(min_combination)


### Fitting an ARIMA Time Series Model

In [ ]:


mod = sm.tsa.statespace.SARIMAX(data,
                                order=min_aic_param,
                                seasonal_order=min_aic_param_seasonal,
                                enforce_stationarity=False,
                                enforce_invertibility=False)

results = mod.fit()


print(results.summary().tables[1])

### Diagnosis of model

In [ ]:
import matplotlib.pyplot as plt

results.plot_diagnostics(figsize=(15, 12))
plt.show()

### Validating Forecasts (Static)

In [ ]:


PREDICTION_START_DATETIME = pd.to_datetime('2015-01-01')

pred = results.get_prediction(start=PREDICTION_START_DATETIME, dynamic=False)
"""
The dynamic=False argument ensures that we produce one-step ahead forecasts, meaning that forecasts at each point are generated using the full history up to that point.
"""

# confidence intervals for forcasts of the time series 
pred_ci = pred.conf_int()



# plotting forcast result
ax = data['2010-01-01':].plot(label='observed')
pred.predicted_mean.plot(ax=ax, label='One-step ahead Forecast', alpha=.7)

ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.2)

ax.fill_betweenx(ax.get_ylim(), PREDICTION_START_DATETIME, data.index[-1],
                 alpha=.1, zorder=-1)


ax.set_xlabel('Date')
ax.set_ylabel('Value')
plt.legend()

plt.show()




### Generating table including all information (Static)

In [ ]:
df_true_value = data.copy()
df_true_value.columns = ['Value']

df_predicted_mean = pd.DataFrame(pred.predicted_mean)
df_predicted_mean.index = pd.to_datetime(df_predicted_mean.index)
df_predicted_mean.index.name = 'DATE'
df_predicted_mean.columns = ['Predicted value']
df_pred_ci = pred_ci.copy()
df_pred_ci.index = pd.to_datetime(df_pred_ci.index)
df_pred_ci.index.name = 'DATE'

#df['Abnomal'] = df[]

#result = left.join(right, how='outer')
df = df_true_value.join(df_predicted_mean, how = 'outer')
df = df.join(df_pred_ci, how = 'outer')

### Calculating MSE, MAPE (Static)

In [ ]:

y_forecasted = df.loc[PREDICTION_START_DATETIME:, 'Predicted value'].values
y_truth = df.loc[PREDICTION_START_DATETIME:, 'Value'].values

# Compute the mean square error
mse = ((y_forecasted - y_truth) ** 2).mean()
print('The Mean Squared Error of our forecasts is {}'.format(round(mse, 2)))

# Compute the mean square error

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape = mean_absolute_percentage_error(y_truth, y_forecasted)
print('The Mean Absolute Percentage Error of our forecasts is {}'.format(round(mape, 2)))

### Validating Forecasts (Dynamic)

In [ ]:
"""Dynamic forecast: only use information from the time series up to a certain point, and after that,
                     forecasts are generated using values from previous forecasted time points.
"""

PREDICTION_START_DATETIME = pd.to_datetime('2015-01-01')

pred_dynamic = results.get_prediction(start=PREDICTION_START_DATETIME, dynamic=True, full_results=True)
pred_dynamic_ci = pred_dynamic.conf_int()

ax = data['2010-01-01':].plot(label='observed')
pred_dynamic.predicted_mean.plot(ax=ax, label='Dynamic Forecast', alpha=.7)

ax.fill_between(pred_dynamic_ci.index,
                pred_dynamic_ci.iloc[:, 0],
                pred_dynamic_ci.iloc[:, 1], color='k', alpha=.25)

ax.fill_betweenx(ax.get_ylim(), PREDICTION_START_DATETIME, data.index[-1],
                 alpha=.1, zorder=-1)

ax.set_xlabel('Date')
ax.set_ylabel('Value')

plt.legend()

plt.show()


### Generating table including all information (Dynamic)

In [ ]:
df_true_value = data.copy()
df_true_value.columns = ['Value']

df_dynamic_predicted_mean = pd.DataFrame(pred_dynamic.predicted_mean)
df_dynamic_predicted_mean.index = pd.to_datetime(df_dynamic_predicted_mean.index)
df_dynamic_predicted_mean.index.name = 'DATE'
df_dynamic_predicted_mean.columns = ['Predicted value']
df_dynamic_pred_ci = pred_dynamic_ci.copy()
df_dynamic_pred_ci.index = pd.to_datetime(df_dynamic_pred_ci.index)
df_dynamic_pred_ci.index.name = 'DATE'

#df['Abnomal'] = df[]

#result = left.join(right, how='outer')
df_dynamic = df_true_value.join(df_dynamic_predicted_mean, how = 'outer')
df_dynamic = df_dynamic.join(df_dynamic_pred_ci, how = 'outer')

### Calculating MSE, MAPE (Dynamic)

In [ ]:
y_forecasted = df_dynamic.loc[PREDICTION_START_DATETIME:, 'Predicted value'].values
y_truth = df_dynamic.loc[PREDICTION_START_DATETIME:, 'Value'].values

# Compute the mean square error
mse = ((y_forecasted - y_truth) ** 2).mean()
print('The Mean Squared Error of our forecasts is {}'.format(round(mse, 2)))

# Compute the mean square error

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape = mean_absolute_percentage_error(y_truth, y_forecasted)
print('The Mean Absolute Percentage Error of our forecasts is {}'.format(round(mape, 2)))

### Future Forecast

In [ ]:
# Get forecast 500 steps ahead in future
pred_uc = results.get_forecast(steps=200)

# Get confidence intervals of forecasts
pred_ci = pred_uc.conf_int()


ax = data.plot(label='observed', figsize=(20, 15))
pred_uc.predicted_mean.plot(ax=ax, label='Forecast')
ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.25)
ax.set_xlabel('Date')
ax.set_ylabel('Value')

plt.legend()
plt.show()

In [ ]:
#from pyramid.arima import auto_arima